# Preprocessing

The preprocessing steps are as follow:
- Read ATL08 points as GDF
- 

TODO:
  - Fix shapely deprecation warning (ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.)

In [3]:
import os
import time
import glob
import logging
import fiona
import rasterio
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxr
import geopandas as gpd
import shapely.speedups
from omegaconf import OmegaConf
from omegaconf.listconfig import ListConfig
from multiprocessing import Pool, cpu_count

# import cuspatial
import tensorflow as tf
from pathlib import Path
from shapely.geometry import box
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error

# from .geoscitools.atl08lib import atl08_io
# from .cnn_model import get_2d_cnn_tf
# from tensorflow_caney.inference.regression_inference import \
#     sliding_window_tiler

shapely.speedups.enable()

## 1. Set Configuration Object

This can be done from a yaml file, or from a dictionary. In here we choose to have a single dictionary for ease of notebook development.

In [11]:
conf = OmegaConf.create(
    {
        "experiment_type": "2m_chm",
        "general_crs": "EPSG:32628",
        "atl08_dir" : "/adapt/nobackup/people/pmontesa/userfs02/data/icesat2/atl08.005/senegal",
        "atl08_start_year": 2018,
        "atl08_end_year": 2022,
        "wv_data_regex" : [
            '/adapt/nobackup/people/mwooten3/Senegal_LCLUC/VHR/CAS/M1BS/*.tif',
            '/adapt/nobackup/people/mwooten3/Senegal_LCLUC/VHR/ETZ/M1BS/*.tif',
            '/adapt/nobackup/people/mwooten3/Senegal_LCLUC/VHR/SRV/M1BS/*.tif'
        ],
        "intersection_output_dir": '/adapt/nobackup/projects/ilab/projects/Senegal/CNN_CHM/intersection_metadata_evhrtoa',
        "tiles_output_dir": '/adapt/nobackup/projects/ilab/projects/Senegal/CNN_CHM/intersection_tiles_evhrtoa',
        "tile_buffer": 520, 
        "model_filename": '/adapt/nobackup/projects/ilab/projects/Senegal/CNN_CHM/model/22-27.97.hdf5',
        "inference_regex": '/adapt/nobackup/people/mcarrol2/LCLUC_Senegal/ForKonrad/Tappan01_WV02_20110430_M1BS_103001000A27E100_data.tif',
        "inference_save_dir": '/adapt/nobackup/people/jacaraba/development/vhr-cnn-chm/data/cnn_chm_v1'
    }
)

## 2. Read ATL08 points

In this section 

In [14]:
def atl08_io(atl08_csv_output_dir, year_search, do_pickle=True):
    '''Read all ATL08 from CSVs of a given year after extract_filter_atl08.py
        Write to a pickle file by year
        Return a geodataframe
    '''
    dir_pickle = atl08_csv_output_dir
    # print("Building list of ATL08 csvs...")

    all_atl08_csvs = glob.glob(
        os.path.join(
            atl08_csv_output_dir, year_search, 'ATL08*100m.csv'
        ), recursive=True
    )

    atl08_gdf = pd.concat(
        (pd.read_csv(f) for f in all_atl08_csvs),
        sort=False, ignore_index=True)  # <--generator is (), list is []

    atl08_gdf = gpd.GeoDataFrame(
        atl08_gdf, geometry=gpd.points_from_xy(
            atl08_gdf.lon, atl08_gdf.lat), crs='epsg:4326')

    if do_pickle:
        # Pickle the file
        if year_search == "**":
            year_search = 'allyears'
        cur_time = time.strftime("%Y%m%d%H%M")
        out_pickle_fn = os.path.join(
            dir_pickle, f"atl08_{year_search}_filt_gdf_{cur_time}.pkl")
        atl08_gdf.to_pickle(out_pickle_fn)
    return atl08_gdf

def get_atl08_gdf(
            data_dir: str,
            start_year: int = 2018,
            end_year: int = 2022,
            reset_index: bool = True,
            crs: str = None
        ):
    """
    Get ATL08 from extracted CSVs.
    """
    # --------------------------------
    # TODO, add GPU component here
    # --------------------------------
    atl08_gdf = pd.concat([
        atl08_io(data_dir, str(year), do_pickle=False)
        for year in range(start_year, end_year)
    ])

    if crs is not None:
        atl08_gdf = atl08_gdf.to_crs(epsg=crs.split(':')[-1])
    return atl08_gdf.reset_index(drop=True)

In [16]:
# Read ATL08 points
atl08_gdf = get_atl08_gdf(
    conf.atl08_dir,
    conf.atl08_start_year,
    conf.atl08_end_year,
    crs=conf.general_crs
)
print(f'Load ATL08 GDF files, {atl08_gdf.shape[0]} rows.')

/gpfsm/ccds01/home/appmgr/app/jupyterhub/ilab/kernel/lib/python3.7/site-packages/geopandas/array.py:275: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.points_from_xy(x, y, z), crs=crs)
/gpfsm/ccds01/home/appmgr/app/jupyterhub/ilab/kernel/lib/python3.7/site-packages/geopandas/array.py:275: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.points_from_xy(x, y, z), crs=crs)
/gpfsm/ccds01/home/appmgr/app/jupyterhub/ilab/kernel/lib/python3.7/site-packages/geopandas/array.py:275: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.points_from_xy(x, y, z), crs=crs)
/gpfsm/ccds01/home/appmgr/app/jupy

Load ATL08 GDF files, 112401 rows.
